In [1]:
import pandas as pd
import numpy as np
from rdkit import DataStructs
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
import openpyxl
import time

#### notebook settings
pd.options.display.max_rows= 100
pd.options.display.max_columns = 50
pd.options.display.max_colwidth = 100

In [2]:
# # @Barbara: uncomment this and modify this to your preferred path
# path_results = "/Users/bzdrazil/Dropbox/ChEMBL/NP/data/"

base_path = "/Users/heinzke/Documents/PhD/Projects/drug_target_dataset_curation/"
path_results = base_path+"results/"

In [3]:
df = pd.read_csv(path_results+"ChEMBL31_DTI_filtered_targets.csv", sep = ";", dtype={'compound_pref_name': 'str'})

In [4]:
len(df)

2378379

In [5]:
df.head()

,Unnamed: 0,molregno,year,journal,pchembl_value,standard_type,assay_type,compound_chembl_id,compound_pref_name,max_phase,first_approval,prodrug,oral,parenteral,topical,black_box_warning,tid,Target_pref_name,target_type,organism,Target_chembl_id,mw_freebase,alogp,hba,hbd,...,molecular_species,full_mwt,aromatic_rings,heavy_atoms,qed_weighted,mw_monoisotopic,full_molformula,hba_lipinski,hbd_lipinski,num_lipinski_ro5_violations,standard_inchi,standard_inchi_key,canonical_smiles,pchembl_value_mean,pchembl_value_max,pchembl_value_median,first_publication_target_cmpd_pair,LE,BEI,SEI,LLE,Therapeutic_Target,defined_DTI,DT_assoc,DTI
0,3,253199,2004.0,Bioorg. Med. Chem. Lett.,5.22,IC50,A,CHEMBL152968,NaN,0,NaN,-1,0,0,0,0,12594,Cytochrome P450 1A2,SINGLE PROTEIN,Homo sapiens,CHEMBL3356,516.67,4.27,7.0,2.0,...,NEUTRAL,516.67,4.0,37.0,0.35,516.2195,C29H32N4O3S,7.0,2.0,1.0,InChI=1S/C29H32N4O3S/c1-21-30-27-17-26(10-11-28(27)37-21)36-20-25(34)18-32-12-14-33(15-13-32)19-...,SZRMSHBRXJWBFG-RUZDIDTESA-N,Cc1nc2cc(OC[C@H](O)CN3CCN(CC(=O)Nc4cccc(-c5ccccc5)c4)CC3)ccc2s1,5.2200,5.22,5.220,2004.0,0.192678,10.103161,6.698319,0.9500,False,False,253199_12594,NDT
1,4,253199,2004.0,Bioorg. Med. Chem. Lett.,4.43,IC50,A,CHEMBL152968,NaN,0,NaN,-1,0,0,0,0,17045,Cytochrome P450 3A4,SINGLE PROTEIN,Homo sapiens,CHEMBL340,516.67,4.27,7.0,2.0,...,NEUTRAL,516.67,4.0,37.0,0.35,516.2195,C29H32N4O3S,7.0,2.0,1.0,InChI=1S/C29H32N4O3S/c1-21-30-27-17-26(10-11-28(27)37-21)36-20-25(34)18-32-12-14-33(15-13-32)19-...,SZRMSHBRXJWBFG-RUZDIDTESA-N,Cc1nc2cc(OC[C@H](O)CN3CCN(CC(=O)Nc4cccc(-c5ccccc5)c4)CC3)ccc2s1,4.4300,4.43,4.430,2004.0,0.163518,8.574138,5.684589,0.1600,True,False,253199_17045,DT
2,5,253199,2004.0,Bioorg. Med. Chem. Lett.,4.62,IC50,A,CHEMBL152968,NaN,0,NaN,-1,0,0,0,0,12911,Cytochrome P450 2C9,SINGLE PROTEIN,Homo sapiens,CHEMBL3397,516.67,4.27,7.0,2.0,...,NEUTRAL,516.67,4.0,37.0,0.35,516.2195,C29H32N4O3S,7.0,2.0,1.0,InChI=1S/C29H32N4O3S/c1-21-30-27-17-26(10-11-28(27)37-21)36-20-25(34)18-32-12-14-33(15-13-32)19-...,SZRMSHBRXJWBFG-RUZDIDTESA-N,Cc1nc2cc(OC[C@H](O)CN3CCN(CC(=O)Nc4cccc(-c5ccccc5)c4)CC3)ccc2s1,4.6200,4.62,4.620,2004.0,0.170531,8.941878,5.928397,0.3500,False,False,253199_12911,NDT
3,7,933,2003.0,J. Med. Chem.,8.19,Ki,B,CHEMBL268439,NaN,0,NaN,-1,0,0,0,0,10193,Carbonic anhydrase I,SINGLE PROTEIN,Homo sapiens,CHEMBL261,335.39,-0.43,8.0,3.0,...,ACID,335.39,2.0,20.0,0.64,334.9817,C8H9N5O4S3,9.0,5.0,0.0,"InChI=1S/C8H9N5O4S3/c9-5-1-3-6(4-2-5)20(16,17)13-7-11-12-8(18-7)19(10,14)15/h1-4H,9H2,(H,11,13)(...",BDLSLORLEPSOGW-UHFFFAOYSA-N,Nc1ccc(S(=O)(=O)Nc2nnc(S(N)(=O)=O)s2)cc1,7.3400,8.22,7.490,2003.0,0.501221,21.884970,4.641750,7.7700,True,False,933_10193,DT
4,8,933,2004.0,J. Med. Chem.,8.69,Ki,B,CHEMBL268439,NaN,0,NaN,-1,0,0,0,0,15,Carbonic anhydrase II,SINGLE PROTEIN,Homo sapiens,CHEMBL205,335.39,-0.43,8.0,3.0,...,ACID,335.39,2.0,20.0,0.64,334.9817,C8H9N5O4S3,9.0,5.0,0.0,"InChI=1S/C8H9N5O4S3/c9-5-1-3-6(4-2-5)20(16,17)13-7-11-12-8(18-7)19(10,14)15/h1-4H,9H2,(H,11,13)(...",BDLSLORLEPSOGW-UHFFFAOYSA-N,Nc1ccc(S(=O)(=O)Nc2nnc(S(N)(=O)=O)s2)cc1,8.1625,8.70,8.305,2004.0,0.557387,24.337339,5.161892,8.5925,True,False,933_15,DT


In [6]:
descriptors_list = [x[0] for x in Descriptors._descList]
print(descriptors_list)

['MaxEStateIndex', 'MinEStateIndex', 'MaxAbsEStateIndex', 'MinAbsEStateIndex', 'qed', 'MolWt', 'HeavyAtomMolWt', 'ExactMolWt', 'NumValenceElectrons', 'NumRadicalElectrons', 'MaxPartialCharge', 'MinPartialCharge', 'MaxAbsPartialCharge', 'MinAbsPartialCharge', 'FpDensityMorgan1', 'FpDensityMorgan2', 'FpDensityMorgan3', 'BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO', 'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW', 'BalabanJ', 'BertzCT', 'Chi0', 'Chi0n', 'Chi0v', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3n', 'Chi3v', 'Chi4n', 'Chi4v', 'HallKierAlpha', 'Ipc', 'Kappa1', 'Kappa2', 'Kappa3', 'LabuteASA', 'PEOE_VSA1', 'PEOE_VSA10', 'PEOE_VSA11', 'PEOE_VSA12', 'PEOE_VSA13', 'PEOE_VSA14', 'PEOE_VSA2', 'PEOE_VSA3', 'PEOE_VSA4', 'PEOE_VSA5', 'PEOE_VSA6', 'PEOE_VSA7', 'PEOE_VSA8', 'PEOE_VSA9', 'SMR_VSA1', 'SMR_VSA10', 'SMR_VSA2', 'SMR_VSA3', 'SMR_VSA4', 'SMR_VSA5', 'SMR_VSA6', 'SMR_VSA7', 'SMR_VSA8', 'SMR_VSA9', 'SlogP_VSA1', 'SlogP_VSA10', 'SlogP_VSA11', 'Slo

In [7]:
#lst = df["canonical_smiles"].to_list()
#lst

In [8]:
calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])

In [9]:
type(calc)

rdkit.ML.Descriptors.MoleculeDescriptors.MolecularDescriptorCalculator

In [10]:
mol = Chem.MolFromSmiles('CC1=CC(=C(C=C1NC(=O)C2=C(C(=CC(=C2)I)I)O)Cl)C(C#N)C3=CC=C(C=C3)Cl')
ds = calc.CalcDescriptors(mol)
print(ds)

(12.745535497884882, -0.5698675869184535, 12.745535497884882, 0.06836196299558472, 0.29235676951522827, 663.0799999999998, 648.9679999999996, 661.852179048, 152, 0, 0.25900360274825185, -0.5060008092821509, 0.5060008092821509, 0.25900360274825185, 1.1333333333333333, 1.8, 2.433333333333333, 126.9137228036413, 9.909169638943874, 2.2033206905749636, -2.1638525434100564, 2.348327336498213, -2.1397445388980523, 14.113673859085004, 0.10192938146936754, 1.9402256061221343, 1175.2355472715938, 21.999271042140983, 15.510685796207323, 21.337540270596875, 14.20561635580521, 8.654060372339206, 11.567487609533984, 6.540209173088069, 9.791667981874944, 4.508130316967467, 6.532560560468579, 2.9673141998441293, 5.210790706092031, -1.5399999999999987, 3810408.214481713, 23.130028268670063, 9.726157452179187, 5.018674352279847, 211.0337664023786, 10.423315998847038, 5.749511833283905, 0.0, 0.0, 5.907179729351506, 0.0, 4.794537184071822, 0.0, 5.261891554738487, 0.0, 41.40098098584986, 99.12766827122579,

In [11]:
# df2 = df
# data = df2
# len(df2)

In [12]:
print('AddMoleculeColumnToFrame')
start = time.process_time()
PandasTools.AddMoleculeColumnToFrame(df,'canonical_smiles','mol',includeFingerprints=False)
print(time.process_time() - start)

# df['mol'] = df.canonical_smiles.apply(Chem.MolFromSmiles)

df.loc[:,'FractionCSP3'] = df['mol'].apply(Descriptors.FractionCSP3)
df.loc[:,'NumAliphaticCarbocycles'] = df['mol'].apply(Descriptors.NumAliphaticCarbocycles)
df.loc[:,'NumAliphaticHeterocycles'] = df['mol'].apply(Descriptors.NumAliphaticHeterocycles)
df.loc[:,'NumAliphaticRings'] = df['mol'].apply(Descriptors.NumAliphaticRings)
df.loc[:,'NumAromaticCarbocycles'] = df['mol'].apply(Descriptors.NumAromaticCarbocycles)
df.loc[:,'NumAromaticHeterocycles'] = df['mol'].apply(Descriptors.NumAromaticHeterocycles)
df.loc[:,'NumAromaticRings'] = df['mol'].apply(Descriptors.NumAromaticRings)
df.loc[:,'NumHeteroatoms'] = df['mol'].apply(Descriptors.NumHeteroatoms)
df.loc[:,'NumSaturatedCarbocycles'] = df['mol'].apply(Descriptors.NumSaturatedCarbocycles)
df.loc[:,'NumSaturatedHeterocycles'] = df['mol'].apply(Descriptors.NumSaturatedHeterocycles)
df.loc[:,'NumSaturatedRings'] = df['mol'].apply(Descriptors.NumSaturatedRings)
df.loc[:,'RingCount'] = df['mol'].apply(Descriptors.RingCount)

AddMoleculeColumnToFrame
201.06194299999999


In [13]:
len(df)

2378379

In [14]:
data_final = df.drop([ 'year', 'journal', 'pchembl_value', 'standard_type', 'Unnamed: 0', 'mol'] , axis=1)

In [15]:
data_final.to_csv(path_results+"ChEMBL31_DTI_filtered_targets_descriptors.csv", sep = ";")

In [16]:
df_final = pd.read_csv(path_results+"ChEMBL31_DTI_filtered_targets_descriptors.csv", sep = ";", dtype={'compound_pref_name': 'str'})

In [17]:
df_final.head()

,Unnamed: 0,molregno,assay_type,compound_chembl_id,compound_pref_name,max_phase,first_approval,prodrug,oral,parenteral,topical,black_box_warning,tid,Target_pref_name,target_type,organism,Target_chembl_id,mw_freebase,alogp,hba,hbd,psa,rtb,ro3_pass,num_ro5_violations,...,canonical_smiles,pchembl_value_mean,pchembl_value_max,pchembl_value_median,first_publication_target_cmpd_pair,LE,BEI,SEI,LLE,Therapeutic_Target,defined_DTI,DT_assoc,DTI,FractionCSP3,NumAliphaticCarbocycles,NumAliphaticHeterocycles,NumAliphaticRings,NumAromaticCarbocycles,NumAromaticHeterocycles,NumAromaticRings,NumHeteroatoms,NumSaturatedCarbocycles,NumSaturatedHeterocycles,NumSaturatedRings,RingCount
0,0,253199,A,CHEMBL152968,NaN,0,NaN,-1,0,0,0,0,12594,Cytochrome P450 1A2,SINGLE PROTEIN,Homo sapiens,CHEMBL3356,516.67,4.27,7.0,2.0,77.93,9.0,N,1.0,...,Cc1nc2cc(OC[C@H](O)CN3CCN(CC(=O)Nc4cccc(-c5ccccc5)c4)CC3)ccc2s1,5.2200,5.22,5.220,2004.0,0.192678,10.103161,6.698319,0.9500,False,False,253199_12594,NDT,0.310345,0,1,1,3,1,4,8,0,1,1,5
1,1,253199,A,CHEMBL152968,NaN,0,NaN,-1,0,0,0,0,17045,Cytochrome P450 3A4,SINGLE PROTEIN,Homo sapiens,CHEMBL340,516.67,4.27,7.0,2.0,77.93,9.0,N,1.0,...,Cc1nc2cc(OC[C@H](O)CN3CCN(CC(=O)Nc4cccc(-c5ccccc5)c4)CC3)ccc2s1,4.4300,4.43,4.430,2004.0,0.163518,8.574138,5.684589,0.1600,True,False,253199_17045,DT,0.310345,0,1,1,3,1,4,8,0,1,1,5
2,2,253199,A,CHEMBL152968,NaN,0,NaN,-1,0,0,0,0,12911,Cytochrome P450 2C9,SINGLE PROTEIN,Homo sapiens,CHEMBL3397,516.67,4.27,7.0,2.0,77.93,9.0,N,1.0,...,Cc1nc2cc(OC[C@H](O)CN3CCN(CC(=O)Nc4cccc(-c5ccccc5)c4)CC3)ccc2s1,4.6200,4.62,4.620,2004.0,0.170531,8.941878,5.928397,0.3500,False,False,253199_12911,NDT,0.310345,0,1,1,3,1,4,8,0,1,1,5
3,3,933,B,CHEMBL268439,NaN,0,NaN,-1,0,0,0,0,10193,Carbonic anhydrase I,SINGLE PROTEIN,Homo sapiens,CHEMBL261,335.39,-0.43,8.0,3.0,158.13,4.0,N,0.0,...,Nc1ccc(S(=O)(=O)Nc2nnc(S(N)(=O)=O)s2)cc1,7.3400,8.22,7.490,2003.0,0.501221,21.884970,4.641750,7.7700,True,False,933_10193,DT,0.000000,0,0,0,1,1,2,12,0,0,0,2
4,4,933,B,CHEMBL268439,NaN,0,NaN,-1,0,0,0,0,15,Carbonic anhydrase II,SINGLE PROTEIN,Homo sapiens,CHEMBL205,335.39,-0.43,8.0,3.0,158.13,4.0,N,0.0,...,Nc1ccc(S(=O)(=O)Nc2nnc(S(N)(=O)=O)s2)cc1,8.1625,8.70,8.305,2004.0,0.557387,24.337339,5.161892,8.5925,True,False,933_15,DT,0.000000,0,0,0,1,1,2,12,0,0,0,2


In [18]:
data_final = df_final

In [19]:
data_final2 = data_final[data_final['max_phase'] > 0]
data_final3 = data_final2[data_final2['DTI'] == 'D_DT']

In [20]:
targets_with_clin_MoA = data_final3.tid.to_list()

In [21]:
data_final_targets_with_clin = data_final.query('tid in @targets_with_clin_MoA')

In [22]:
print(data_final_targets_with_clin.tid.nunique())

369


In [23]:
print(data_final_targets_with_clin.compound_chembl_id.nunique())

430051


In [24]:
data_final_targets_with_clin.head()

,Unnamed: 0,molregno,assay_type,compound_chembl_id,compound_pref_name,max_phase,first_approval,prodrug,oral,parenteral,topical,black_box_warning,tid,Target_pref_name,target_type,organism,Target_chembl_id,mw_freebase,alogp,hba,hbd,psa,rtb,ro3_pass,num_ro5_violations,...,canonical_smiles,pchembl_value_mean,pchembl_value_max,pchembl_value_median,first_publication_target_cmpd_pair,LE,BEI,SEI,LLE,Therapeutic_Target,defined_DTI,DT_assoc,DTI,FractionCSP3,NumAliphaticCarbocycles,NumAliphaticHeterocycles,NumAliphaticRings,NumAromaticCarbocycles,NumAromaticHeterocycles,NumAromaticRings,NumHeteroatoms,NumSaturatedCarbocycles,NumSaturatedHeterocycles,NumSaturatedRings,RingCount
1,1,253199,A,CHEMBL152968,NaN,0,NaN,-1,0,0,0,0,17045,Cytochrome P450 3A4,SINGLE PROTEIN,Homo sapiens,CHEMBL340,516.67,4.27,7.0,2.0,77.93,9.0,N,1.0,...,Cc1nc2cc(OC[C@H](O)CN3CCN(CC(=O)Nc4cccc(-c5ccccc5)c4)CC3)ccc2s1,4.4300,4.43,4.430,2004.0,0.163518,8.574138,5.684589,0.1600,True,False,253199_17045,DT,0.310345,0,1,1,3,1,4,8,0,1,1,5
3,3,933,B,CHEMBL268439,NaN,0,NaN,-1,0,0,0,0,10193,Carbonic anhydrase I,SINGLE PROTEIN,Homo sapiens,CHEMBL261,335.39,-0.43,8.0,3.0,158.13,4.0,N,0.0,...,Nc1ccc(S(=O)(=O)Nc2nnc(S(N)(=O)=O)s2)cc1,7.3400,8.22,7.490,2003.0,0.501221,21.884970,4.641750,7.7700,True,False,933_10193,DT,0.000000,0,0,0,1,1,2,12,0,0,0,2
4,4,933,B,CHEMBL268439,NaN,0,NaN,-1,0,0,0,0,15,Carbonic anhydrase II,SINGLE PROTEIN,Homo sapiens,CHEMBL205,335.39,-0.43,8.0,3.0,158.13,4.0,N,0.0,...,Nc1ccc(S(=O)(=O)Nc2nnc(S(N)(=O)=O)s2)cc1,8.1625,8.70,8.305,2004.0,0.557387,24.337339,5.161892,8.5925,True,False,933_15,DT,0.000000,0,0,0,1,1,2,12,0,0,0,2
7,7,933,B,CHEMBL268439,NaN,0,NaN,-1,0,0,0,0,12209,Carbonic anhydrase XII,SINGLE PROTEIN,Homo sapiens,CHEMBL3242,335.39,-0.43,8.0,3.0,158.13,4.0,N,0.0,...,Nc1ccc(S(=O)(=O)Nc2nnc(S(N)(=O)=O)s2)cc1,7.0700,7.07,7.070,2005.0,0.482784,21.079937,4.471005,7.5000,True,False,933_12209,DT,0.000000,0,0,0,1,1,2,12,0,0,0,2
14,14,933,B,CHEMBL268439,NaN,0,NaN,-1,0,0,0,0,10193,Carbonic anhydrase I,SINGLE PROTEIN,Homo sapiens,CHEMBL261,335.39,-0.43,8.0,3.0,158.13,4.0,N,0.0,...,Nc1ccc(S(=O)(=O)Nc2nnc(S(N)(=O)=O)s2)cc1,7.3400,8.22,7.490,2003.0,0.501221,21.884970,4.641750,7.7700,True,False,933_10193,DT,0.000000,0,0,0,1,1,2,12,0,0,0,2


In [25]:
data_final_targets_with_clin.drop(['Unnamed: 0'] , axis=1)

,molregno,assay_type,compound_chembl_id,compound_pref_name,max_phase,first_approval,prodrug,oral,parenteral,topical,black_box_warning,tid,Target_pref_name,target_type,organism,Target_chembl_id,mw_freebase,alogp,hba,hbd,psa,rtb,ro3_pass,num_ro5_violations,cx_most_apka,...,canonical_smiles,pchembl_value_mean,pchembl_value_max,pchembl_value_median,first_publication_target_cmpd_pair,LE,BEI,SEI,LLE,Therapeutic_Target,defined_DTI,DT_assoc,DTI,FractionCSP3,NumAliphaticCarbocycles,NumAliphaticHeterocycles,NumAliphaticRings,NumAromaticCarbocycles,NumAromaticHeterocycles,NumAromaticRings,NumHeteroatoms,NumSaturatedCarbocycles,NumSaturatedHeterocycles,NumSaturatedRings,RingCount
1,253199,A,CHEMBL152968,NaN,0,NaN,-1,0,0,0,0,17045,Cytochrome P450 3A4,SINGLE PROTEIN,Homo sapiens,CHEMBL340,516.67,4.27,7.0,2.0,77.93,9.0,N,1.0,13.29,...,Cc1nc2cc(OC[C@H](O)CN3CCN(CC(=O)Nc4cccc(-c5ccccc5)c4)CC3)ccc2s1,4.4300,4.43,4.430,2004.0,0.163518,8.574138,5.684589,0.1600,True,False,253199_17045,DT,0.310345,0,1,1,3,1,4,8,0,1,1,5
3,933,B,CHEMBL268439,NaN,0,NaN,-1,0,0,0,0,10193,Carbonic anhydrase I,SINGLE PROTEIN,Homo sapiens,CHEMBL261,335.39,-0.43,8.0,3.0,158.13,4.0,N,0.0,6.48,...,Nc1ccc(S(=O)(=O)Nc2nnc(S(N)(=O)=O)s2)cc1,7.3400,8.22,7.490,2003.0,0.501221,21.884970,4.641750,7.7700,True,False,933_10193,DT,0.000000,0,0,0,1,1,2,12,0,0,0,2
4,933,B,CHEMBL268439,NaN,0,NaN,-1,0,0,0,0,15,Carbonic anhydrase II,SINGLE PROTEIN,Homo sapiens,CHEMBL205,335.39,-0.43,8.0,3.0,158.13,4.0,N,0.0,6.48,...,Nc1ccc(S(=O)(=O)Nc2nnc(S(N)(=O)=O)s2)cc1,8.1625,8.70,8.305,2004.0,0.557387,24.337339,5.161892,8.5925,True,False,933_15,DT,0.000000,0,0,0,1,1,2,12,0,0,0,2
7,933,B,CHEMBL268439,NaN,0,NaN,-1,0,0,0,0,12209,Carbonic anhydrase XII,SINGLE PROTEIN,Homo sapiens,CHEMBL3242,335.39,-0.43,8.0,3.0,158.13,4.0,N,0.0,6.48,...,Nc1ccc(S(=O)(=O)Nc2nnc(S(N)(=O)=O)s2)cc1,7.0700,7.07,7.070,2005.0,0.482784,21.079937,4.471005,7.5000,True,False,933_12209,DT,0.000000,0,0,0,1,1,2,12,0,0,0,2
14,933,B,CHEMBL268439,NaN,0,NaN,-1,0,0,0,0,10193,Carbonic anhydrase I,SINGLE PROTEIN,Homo sapiens,CHEMBL261,335.39,-0.43,8.0,3.0,158.13,4.0,N,0.0,6.48,...,Nc1ccc(S(=O)(=O)Nc2nnc(S(N)(=O)=O)s2)cc1,7.3400,8.22,7.490,2003.0,0.501221,21.884970,4.641750,7.7700,True,False,933_10193,DT,0.000000,0,0,0,1,1,2,12,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2378369,2396782,B,CHEMBL4526351,BAY-309,0,NaN,-1,0,0,0,0,101079,Discoidin domain-containing receptor 2,SINGLE PROTEIN,Homo sapiens,CHEMBL5122,519.50,7.41,5.0,1.0,64.22,5.0,N,2.0,NaN,...,Cc1cc(NC(=O)c2cccc(S(F)(F)(F)(F)F)c2)cc(-n2ccn3nc(-c4cccnc4)cc23)c1,7.2000,7.20,7.200,2021.0,0.273145,13.859480,11.211461,-0.2100,True,False,2396782_101079,DT,0.041667,0,0,0,2,3,5,12,0,0,0,5
2378370,2408605,B,CHEMBL4538174,ERKi,0,NaN,-1,0,0,0,0,12829,Cyclin-dependent kinase 7,SINGLE PROTEIN,Homo sapiens,CHEMBL3055,311.35,2.33,4.0,4.0,102.93,4.0,N,0.0,9.27,...,C[C@@H](NC(=O)Nc1cc2[nH]ncc2c(CO)n1)c1ccccc1,6.6400,6.64,6.640,2021.0,0.394279,21.326481,6.450986,4.3100,True,False,2408605_12829,DT,0.187500,0,0,0,1,2,3,7,0,0,0,3
2378372,2408605,B,CHEMBL4538174,ERKi,0,NaN,-1,0,0,0,0,11639,MAP kinase ERK1,SINGLE PROTEIN,Homo sapiens,CHEMBL3385,311.35,2.33,4.0,4.0,102.93,4.0,N,0.0,9.27,...,C[C@@H](NC(=O)Nc1cc2[nH]ncc2c(CO)n1)c1ccccc1,8.8200,8.82,8.820,2021.0,0.523726,28.328248,8.568930,6.4900,True,False,2408605_11639,DT,0.187500,0,0,0,1,2,3,7,0,0,0,3
2378373,2408605,B,CHEMBL4538174,ERKi,0,NaN,-1,0,0,0,0,11638,MAP kinase ERK2,SINGLE PROTEIN,Homo sapiens,CHEMBL4040,311.35,2.33,4.0,4.0,102.93,4.0,N,0.0,9.27,...,C[C@@H](NC(=O)Nc1cc2[nH]ncc2c(CO)n1)c1ccccc1,8.3200,8.32,8.320,2021.0,0.494036,26.722338,8.083163,5.9900,True,False,2408605_11638,DT,0.187500,0,0,0,1,2,3,7,0,0,0,3


In [26]:
data_final_targets_with_clin.to_csv(path_results+"ChEMBL31_DTI_targets_with_clin_descriptors.csv", sep = ";")

In [27]:
print(len(data_final_targets_with_clin))

748107


In [28]:
data_final_targets_with_clin_4 = data_final_targets_with_clin[data_final_targets_with_clin['max_phase'] == 4]
data_final_targets_with_clin_4_D_DT = data_final_targets_with_clin_4[data_final_targets_with_clin_4['DTI'] == 'D_DT']
data_final_targets_with_clin_4_D_DT.tid.nunique() 

193

In [29]:
data_final_targets_with_clin_3 = data_final_targets_with_clin[data_final_targets_with_clin['max_phase'] == 3]
data_final_targets_with_clin_3_D_DT = data_final_targets_with_clin_3[data_final_targets_with_clin_3['DTI'] == 'D_DT']
data_final_targets_with_clin_3_D_DT.tid.nunique() 

164

In [30]:
data_final_targets_with_clin_2 = data_final_targets_with_clin[data_final_targets_with_clin['max_phase'] == 2]
data_final_targets_with_clin_2_D_DT = data_final_targets_with_clin_2[data_final_targets_with_clin_2['DTI'] == 'D_DT']
data_final_targets_with_clin_2_D_DT.tid.nunique() 

220

In [31]:
data_final_targets_with_clin_1 = data_final_targets_with_clin[data_final_targets_with_clin['max_phase'] == 1]
data_final_targets_with_clin_1_D_DT = data_final_targets_with_clin_1[data_final_targets_with_clin_1['DTI'] == 'D_DT']
data_final_targets_with_clin_1_D_DT.tid.nunique() 

70

In [32]:
data_final_targets_with_clin.to_excel(path_results+"ChEMBL31_DTI_targets_with_clin_descriptors.xlsx")